In [1]:
from glob import glob
from datetime import datetime
import numpy as np
import traceback

npzfiles = list(glob('*.npz'))
npzfiles.sort()

In [2]:
import multiprocessing as mp
import re

def check_file_status(filename):
    filenumber = re.findall(r'CTPMHg_results\.([0-9]+)\.npz', filename)[0]
    npzfile = np.load(filename)
    
    with open('../notebook_logs/file_status.{}.log'.format(filenumber), 'a') as file_pointer:
        try:
            assert npzfile.files == ['pop_sizes', 'sample_sizes']
            file_pointer.write('File #{} has the expected variables.\n'.format(filenumber))
            pop_sizes = npzfile['pop_sizes']
            sample_sizes = npzfile['sample_sizes']
            try:
                assert pop_sizes.shape == (91, 15000000)
                file_pointer.write('Variable `pop_sizes` has expected shape for file #{}.\n'.format(filenumber))
                try:
                    assert sample_sizes.shape == (91, 15000000)
                    file_pointer.write('Variable `sample_sizes` has expected shape for file #{}.\n'.format(filenumber))
                    file_pointer.write('All checks for file #{} passed.\n'.format(filenumber))
                except AssertionError:
                    file_pointer.write('Variable `sample_sizes` does NOT have expected shape for file #{}.\n'.format(filenumber))
                    file_pointer.write('Checks for file #{} failed.\n'.format(filenumber))
            except AssertionError:
                file_pointer.write('Variable `pop_sizes` does NOT have expected shape for file #{}.\n'.format(filenumber))
                file_pointer.write('Checks for file #{} failed.\n'.format(filenumber))
                try:
                    assert sample_sizes.shape == (91, 15000000)
                    file_pointer.write('Variable `sample_sizes` has expected shape for file #{}.\n'.format(filenumber))
                except AssertionError:
                    file_pointer.write('Variable `sample_sizes` does NOT have expected shape for file #{}.\n'.format(filenumber))
                    file_pointer.write('Checks for file #{} failed.\n'.format(filenumber))                
        except AssertionError:
            file_pointer.write('File #{} does NOT have the expected variables.\n'.format(filenumber))
            file_pointer.write('Checks for file #{} failed.\n'.format(filenumber))

In [3]:
%%time
try:
    pool = mp.Pool(40)
    pool.map(check_file_status, npzfiles)
    pool.close()
    pool.join()
except RuntimeError as error_message:
    timestamp_filename = datetime.now().strftime('%m-%d-%Y.at.%H_%M_%S.log')
    with open('../error_logs/{}'.format(timestamp_filename), 'a') as file_pointer:
        file_pointer.write(str(error_message))
        file_pointer.write(traceback.format_exc())

CPU times: user 360 ms, sys: 991 ms, total: 1.35 s
Wall time: 27min 41s
